## download dataset stright from kaggle

In [ ]:
%%capture
%pip install kaggle

#upload your tokn
from google.colab import files
import time

uploaded = files.upload()
time.sleep(3)

#download directly from kaggle
%cp kaggle.json ~/.kaggle/
%chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download "dogs-vs-cats"

## unzip dataset


In [ ]:
import zipfile

#root directory
data_zip = "/content/dogs-vs-cats.zip"
data_dir = "./data"
data_zip_ref = zipfile.ZipFile(data_zip,"r")
data_zip_ref.extractall(data_dir)

#test
test_zip = "/content/data/test1.zip"
test_dir = "./data"
test_zip_ref = zipfile.ZipFile(test_zip,"r")
test_zip_ref.extractall(test_dir)

#train
train_zip = "/content/data/train.zip"
train_dir = "./data"
train_zip_ref = zipfile.ZipFile(train_zip,"r")
train_zip_ref.extractall(train_dir)

## Structure and populate the subfolders

In [ ]:
import os
import glob

# dat_dir = "/content/data"
data_dir = "./"

#create training dir
training_dir = os.path.join(data_dir,"training")
if not os.path.isdir(training_dir):
  os.mkdir(training_dir)

#create dog in training
dog_training_dir = os.path.join(training_dir,"dog")
if not os.path.isdir(dog_training_dir):
  os.mkdir(dog_training_dir)

#create cat in training
cat_training_dir = os.path.join(training_dir,"cat")
if not os.path.isdir(cat_training_dir):
  os.mkdir(cat_training_dir)

#create validation dir
validation_dir = os.path.join(data_dir,"validation")
if not os.path.isdir(validation_dir):
  os.mkdir(validation_dir)

#create dog in validation
dog_validation_dir = os.path.join(validation_dir,"dog")
if not os.path.isdir(dog_validation_dir):
  os.mkdir(dog_validation_dir)

#create cat in validation
cat_validation_dir = os.path.join(validation_dir,"cat")
if not os.path.isdir(cat_validation_dir):
  os.mkdir(cat_validation_dir)

### Shuffle newly aquired data in folder

In [ ]:
import shutil
from numpy import size

split_size = 0.80
# cat_imgs_size = len(glob.glob("/content/data/train/cat*"))
# dog_imgs_size = len(glob.glob("/content/data/train/dog*"))
cat_imgs_size = len(glob.glob("./archive (1)/train/train/cat*"))
dog_imgs_size = len(glob.glob("./archive (1)/train/train/dog*"))


size = 300


cat_counter = size
for i,img in enumerate(glob.glob("./archive (1)/train/train/cat*")):
  if(cat_counter==0):
    break
  if i < (size * split_size):
    shutil.move(img,cat_training_dir)
    cat_counter -= 1
  else:
    shutil.move(img,cat_validation_dir)
    cat_counter -= 1


dog_counter = size
for i,img in enumerate(glob.glob("./archive (1)/train/train/dog*")):
  if(dog_counter==0):
    break
  if i < (size * split_size):
    shutil.move(img,dog_training_dir)
    dog_counter -= 1
  else:
    shutil.move(img,dog_validation_dir)
    dog_counter -= 1

## plot some examples


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
# import cv2

from IPython.core.pylabtools import figsize

samples_dog = [os.path.join(dog_training_dir,np.random.choice(a = os.listdir(dog_training_dir),size = 1)[0]) for _ in range(8)]
samples_cat = [os.path.join(cat_training_dir,np.random.choice(a = os.listdir(cat_training_dir),size = 1)[0]) for _ in range(8)]

nrows = 4
ncols = 4

fig, ax = plt.subplots(nrows,ncols,figsize = (10,10))
ax = ax.flatten()

for i in range(nrows*ncols):
  if i < 8:
    pic = plt.imread(samples_dog[i%8])
    ax[i].imshow(pic)
    ax[i].set_axis_off()
  else:
    pic = plt.imread(samples_cat[i%8])
    ax[i].imshow(pic)
    ax[i].set_axis_off()
plt.show()

## Create dataloader

Now we are going to do 3 things:

1. Let’s preprocess our data using the compose method, which is a simple method to apply multiple preprocessing functions like normalization and data augmentation to our dataset.
2. Let’s use ImageFolder to create a pytorch dataset. PyTorch infers the class automatically if the subdirectories structure is well defined (as in our case).
3. Use the DataLoader to slice our data in batches.

In [ ]:
import torch
import torch.utils.data
import torchvision
from torchvision import datasets, transforms

# traindir = "/content/data/training"
# testdir = "/content/data/validation"
traindir = "./training"
testdir = "./validation"

#transformations
train_transforms = transforms.Compose([transforms.Resize((224,224)),
                                       transforms.ToTensor(),
                                       torchvision.transforms.Normalize(
                                           mean=[0.485, 0.456, 0.406],
                                           std=[0.229, 0.224, 0.225],
    ),
                                       ])
test_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      torchvision.transforms.Normalize(
                                          mean=[0.485, 0.456, 0.406],
                                          std=[0.229, 0.224, 0.225],
    ),
                                      ])

#datasets
train_data = datasets.ImageFolder(traindir,transform=train_transforms)
test_data = datasets.ImageFolder(testdir,transform=test_transforms)

#dataloader
trainloader = torch.utils.data.DataLoader(train_data, shuffle = True, batch_size=16)
testloader = torch.utils.data.DataLoader(test_data, shuffle = True, batch_size=16)

## training function

In [ ]:
def make_train_step(model, optimizer, loss_fn):
  def train_step(x,y):
    #make prediction
    yhat = model(x) #forward pass
    #enter train mode
    model.train() # make model to train
    #compute loss
    loss = loss_fn(yhat,y) # calculate loss

    loss.backward() # backward propogation
    optimizer.step() #optimizer step
    optimizer.zero_grad() 
    #optimizer.cleargrads()

    return loss
  return train_step

## Our model used

In [ ]:
from torchvision import datasets, models, transforms
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.resnet18(pretrained=True)

#freeze all params
# for params in model.parameters():
#   params.requires_grad_ = False

model.eval()
#add a new final layer
with torch.no_grad():
  nr_filters = model.fc.in_features  #number of input features of last layer
  model.fc = nn.Linear(nr_filters, 1)

  model = model.to(device)

### Optimizer and make train step

In [ ]:
from torch.nn.modules.loss import BCEWithLogitsLoss
from torch.optim import lr_scheduler

#loss
loss_fn = BCEWithLogitsLoss() #binary cross entropy with sigmoid, so no need to use sigmoid in the model

#optimizer
optimizer = torch.optim.Adam(model.fc.parameters())

#train step
train_step = make_train_step(model, optimizer, loss_fn)

## Train the model


In [ ]:
# %%capture
# %pip install tqdm
from tqdm import tqdm


losses = []
val_losses = []

epoch_train_losses = []
epoch_test_losses = []

n_epochs = 3 #10
early_stopping_tolerance = 3
early_stopping_threshold = 0.3 # 0.03

for epoch in range(n_epochs):
  print(f"epoch {epoch} is running")
  epoch_loss = 0
  for i ,data in tqdm(enumerate(trainloader), total = len(trainloader)): #iterate ove batches
  # for i ,data in enumerate(trainloader): #iterate ove batches
    x_batch , y_batch = data
    x_batch = x_batch.to(device) #move to gpu
    y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
    y_batch = y_batch.to(device) #move to gpu

    loss = train_step(x_batch, y_batch)
    epoch_loss += loss/len(trainloader)
    losses.append(loss)

  epoch_train_losses.append(epoch_loss)
  print('\nEpoch : {}, train loss : {}'.format(epoch+1,epoch_loss))

  #validation doesnt requires gradient
  with torch.no_grad():
    cum_loss = 0
    for x_batch, y_batch in testloader:
      x_batch = x_batch.to(device)
      y_batch = y_batch.unsqueeze(1).float() #convert target to same nn output shape
      y_batch = y_batch.to(device)

      #model to eval mode
      model.eval()

      yhat = model(x_batch)
      val_loss = loss_fn(yhat,y_batch)
      cum_loss += loss/len(testloader)
      val_losses.append(val_loss.item())


    epoch_test_losses.append(cum_loss)
    print('Epoch : {}, val loss : {}'.format(epoch+1,cum_loss))

    best_loss = min(epoch_test_losses)

    #save best model
    if cum_loss <= best_loss:
      best_model_wts = model.state_dict()

    #early stopping
    early_stopping_counter = 0
    if cum_loss > best_loss:
      early_stopping_counter +=1

    if (early_stopping_counter == early_stopping_tolerance) or (best_loss <= early_stopping_threshold):
      print("/nTerminating: early stopping")
      break #terminate training

#load best model
model.load_state_dict(best_model_wts)

## Inference

In [ ]:
import matplotlib.pyplot as plt

def inference(test_data):
  idx = torch.randint(1, len(test_data), (1,))
  sample = torch.unsqueeze(test_data[idx][0], dim=0).to(device)

  if torch.sigmoid(model(sample)) < 0.5:
    print("Prediction : Cat")
  else:
    print("Prediction : Dog")


  plt.imshow(test_data[idx][0].permute(1, 2, 0))

inference(test_data)